In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_size = (224, 224)
batch_size = 32

In [ ]:
# train dataset

train_ds = tf.keras.utils.image_dataset_from_directory(
    "Images",
    validation_split=0.2, # 80/20 split
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 20580 files belonging to 120 classes.
Using 16464 files for training.


In [ ]:
# validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    "Images",
    validation_split=0.2, # 80/20 split
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [ ]:

class_names = train_ds.class_names
num_classes = len(class_names)
print("Number of breeds:", num_classes)

Number of breeds: 120


# Step 3: Data Augmentation + Prefetch\

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.prefetch(buffer_size=AUTOTUNE)


In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
base_model.trainable = False  # freeze backbone for transfer learning

In [ ]:
inputs = layers.Input(shape=img_size + (3,))

In [ ]:
x = tf.keras.applications.resnet50.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)

In [ ]:
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Step 5: Train

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 256s 469ms/step - accuracy: 0.0706 - loss: 4.6093 - val_accuracy: 0.5046 - val_loss: 2.2767
Epoch 2/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 226s 438ms/step - accuracy: 0.3969 - loss: 2.4860 - val_accuracy: 0.6601 - val_loss: 1.4322
Epoch 3/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 225s 436ms/step - accuracy: 0.5497 - loss: 1.7459 - val_accuracy: 0.7150 - val_loss: 1.1216
Epoch 4/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 230s 446ms/step - accuracy: 0.6379 - loss: 1.4042 - val_accuracy: 0.7357 - val_loss: 0.9703
Epoch 5/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 263s 447ms/step - accuracy: 0.6769 - loss: 1.2028 - val_accuracy: 0.7505 - val_loss: 0.8841
Epoch 6/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 257s 437ms/step - accuracy: 0.6937 - loss: 1.0980 - val_accuracy: 0.7573 - val_loss: 0.8315
Epoch 7/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 226s 438ms/step - accuracy: 0.7227 - loss: 0.9897 - val_accuracy: 0.7721 - val_loss: 0.7883
Epoch 8/10
515/515 ━━━━━━━━━━━━━━━━━━━━ 226s 439ms/step - accuracy: 0.7408 -

In [ ]:
base_model.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to image you want to test
img_path = "/content/drive/MyDrive/Colab Notebooks/Project Unique/Dog test/hero_belgian_malinois.avif"

# Trying a more common path for mounted Google Drive files
#img_path = "/content/drive/MyDrive/GR.webp"


# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # shape (1, 224, 224, 3)

# Use preprocess_input if using a pretrained backbone
img_array = tf.keras.applications.resnet50.preprocess_input(img_array) # Use resnet50 preprocess_input

# Predict
predictions = model.predict(img_array)  # shape (1, num_classes)
predicted_class_index = np.argmax(predictions, axis=1)[0]
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted Breed: {predicted_class_name}")

In [ ]:
model.save("/content/drive/MyDrive/stanford_dogs_model.keras")


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/stanford_dogs_model.keras")


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("/content/drive/MyDrive/best_model.keras",
                             monitor="val_accuracy",
                             save_best_only=True,
                             mode="max")

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10,
                    callbacks=[checkpoint])

# For making classes

In [ ]:
import tarfile
import os

tar_path = "/content/drive/MyDrive/Colab Notebooks/Project Unique/Dog Dataset/images.tar"
extract_dir = "/content/drive/MyDrive/Colab Notebooks/Project Unique/Dog Dataset/images"  # where to extract

# Create target directory if not exists
os.makedirs(extract_dir, exist_ok=True)

# Extract
with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extract_dir)

print("✅ Extracted to:", extract_dir)


In [ ]:
import os

dataset_dir = "/content/drive/MyDrive/Colab Notebooks/Project Unique/Dog Dataset/images/Images"
folders = sorted(os.listdir(dataset_dir))  # sorted to match training order
class_names = []

for f in folders:
    if os.path.isdir(os.path.join(dataset_dir, f)):  # ensure it's a folder
        name = f.split('-', 1)[-1].replace('_', ' ')
        class_names.append(name)

print("Total classes:", len(class_names))
print("Sample:", class_names[:10])


Total classes: 120
Sample: ['Chihuahua', 'Japanese spaniel', 'Maltese dog', 'Pekinese', 'Shih-Tzu', 'Blenheim spaniel', 'papillon', 'toy terrier', 'Rhodesian ridgeback', 'Afghan hound']


In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/Project Unique/standfordDogClasses.txt", "w") as f:
    for name in class_names:
        f.write(name + "\n")


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --------------------------
# Install packages
# --------------------------
!pip install ultralytics opencv-python-headless matplotlib tensorflow

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
from ultralytics import YOLO
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

# --------------------------
# 1. Load Models
# --------------------------
detector = YOLO("yolov8n.pt")  # YOLOv8 object detector (for dog detection)
breed_classifier_model = load_model("/content/drive/MyDrive/stanford_dogs_model.keras")

# Load your custom breed class names (must match training order)
with open("/content/drive/MyDrive/Colab Notebooks/Project Unique/standfordDogClasses.txt") as f:
    class_names = [line.strip() for line in f]

# --------------------------
# 2. Download or Load a Sample Dog Image
# --------------------------
url = "https://static.vecteezy.com/system/resources/previews/005/857/332/non_2x/funny-portrait-of-cute-corgi-dog-outdoors-free-photo.jpg"
img_path = "/content/drive/MyDrive/Colab Notebooks/Project Unique/Dog test/zs.webp"

if not os.path.exists(img_path):
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(img_path, "wb") as f:
        f.write(r.content)

# --------------------------
# 3. Run YOLO Detection
# --------------------------
try:
    img_pil = Image.open(img_path).convert("RGB")  # Ensure RGB
    img_array = np.array(img_pil)
    results = detector(img_array)  # Run YOLO detection
except Exception as e:
    print(f"Error loading or processing image: {e}")
    results = None

# --------------------------
# 4. Crop Dog Regions
# --------------------------
dog_crops = []
if results and results[0].boxes is not None:
    boxes = results[0].boxes.xyxy.cpu().numpy()   # Bounding boxes (x1,y1,x2,y2)
    labels = results[0].boxes.cls.cpu().numpy()   # Class labels
    img_cv = cv2.imread(img_path)                 # Load for cropping (BGR)

    for i, box in enumerate(boxes):
        if int(labels[i]) == 16:  # COCO class 16 = dog
            x1, y1, x2, y2 = map(int, box)
            h, w, _ = img_cv.shape
            x1, y1, x2, y2 = max(0,x1), max(0,y1), min(w,x2), min(h,y2)
            if x2 > x1 and y2 > y1:
                dog_crop = img_cv[y1:y2, x1:x2]
                dog_crops.append(dog_crop)
else:
    print("No dogs detected.")

# --------------------------
# 5. Breed Classification
# --------------------------
predictions = []
if dog_crops:
    for i, crop in enumerate(dog_crops):
        crop_path = f"dog_crop_{i}.jpg"
        cv2.imwrite(crop_path, crop)

        try:
            img = image.load_img(crop_path, target_size=(224,224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)  # Same preprocessing used in training

            preds = breed_classifier_model.predict(x)  # (1, num_classes)
            probs = preds[0]
            top_indices = probs.argsort()[-3:][::-1]  # Top 3 predictions
            decoded = [(class_names[j], float(probs[j])) for j in top_indices]
            predictions.append(decoded)
        except Exception as e:
            print(f"Error classifying dog crop {i}: {e}")
            predictions.append(f"Classification Error: {e}")

# --------------------------
# 6. Display Results
# --------------------------
if results:
    res_plot = results[0].plot()
    plt.figure(figsize=(10,5))
    plt.subplot(1, max(1, len(dog_crops) + (1 if dog_crops else 0)), 1)
    plt.imshow(cv2.cvtColor(res_plot, cv2.COLOR_BGR2RGB))
    plt.title("Detected Dog(s)")
    plt.axis("off")

if dog_crops:
    for i, crop in enumerate(dog_crops):
        plt.subplot(1, max(1, len(dog_crops) + (1 if results else 0)), i + (2 if results else 1))
        plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.title(f"Cropped Dog {i+1}")
        plt.axis("off")

plt.tight_layout()
plt.show()

# --------------------------
# 7. Print Breed Predictions
# --------------------------
if predictions:
    for i, decoded in enumerate(predictions):
        print(f"🐕 Dog {i+1} Breed Predictions:")
        if isinstance(decoded, list):
            for (breed, score) in decoded:
                print(f" - {breed} ({score*100:.2f}%)")
        else:
            print(decoded)


# ✅  Wrap into a function / API

Instead of running everything in a notebook cell, wrap the pipeline into a function:

In [ ]:
import os
import numpy as np
import cv2
from flask import Flask, request, jsonify
from PIL import Image
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input  # Add this import
import tensorflow as tf

print(tf.__version__)

# ----------------------------
# [INFO] Load models
# ----------------------------
print("[INFO] Loading YOLO model...")
yolo = YOLO("yolov8n.pt")

print("[INFO] Loading custom breed model...")
classifier = load_model("/content/drive/MyDrive/stanford_dogs_model.keras")

with open("/content/drive/MyDrive/Colab Notebooks/Project Unique/standfordDogClasses.txt") as f:
    BREED_CLASSES = [line.strip() for line in f]

# ----------------------------
# Helper function
# ----------------------------
def predict_breed(img_path):
    try:
        # Load image as PIL Image first (like in working code)
        img_pil = Image.open(img_path).convert("RGB")
        img_array = np.array(img_pil)

        # Detect dog with YOLO
        results = yolo(img_array)

        # Check if any dogs were detected
        if results[0].boxes is None:
            return {"error": "No dogs detected"}

        boxes = results[0].boxes.xyxy.cpu().numpy()
        labels = results[0].boxes.cls.cpu().numpy()

        # Find the first dog (COCO class 16)
        dog_box = None
        for i, box in enumerate(boxes):
            if int(labels[i]) == 16:  # COCO class 16 = dog
                dog_box = box
                break

        if dog_box is None:
            return {"error": "No dog detected"}

        # Crop the dog region
        x1, y1, x2, y2 = map(int, dog_box)

        # Load image with OpenCV for cropping (like in working code)
        img_cv = cv2.imread(img_path)
        h, w, _ = img_cv.shape

        # Ensure coordinates are within bounds
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

        if x2 <= x1 or y2 <= y1:
            return {"error": "Invalid crop dimensions"}

        dog_crop = img_cv[y1:y2, x1:x2]

        # Save cropped image temporarily
        crop_path = "temp_crop.jpg"
        cv2.imwrite(crop_path, dog_crop)

        # Load and preprocess exactly like in working code
        img = image.load_img(crop_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)  # Use ResNet50 preprocessing instead of /255.0

        # Predict
        preds = classifier.predict(x)

        # Get top 3 predictions like in working code
        probs = preds[0]
        top_indices = probs.argsort()[-3:][::-1]  # Top 3 predictions

        top_predictions = []
        for idx in top_indices:
            breed_name = BREED_CLASSES[idx]
            confidence = float(probs[idx])
            top_predictions.append({
                "breed": breed_name,
                "confidence": confidence
            })

        print("Check top predictions---------->", top_predictions)

        # Clean up temporary file
        if os.path.exists(crop_path):
            os.remove(crop_path)

        #return {
        #    "top_prediction": top_predictions[0],
        #    "all_predictions": top_predictions
        #}
        return {
            "breed": top_predictions[0]["breed"],
            "confidence": top_predictions[0]["confidence"]
        }
    except Exception as e:
        print(f"Error in predict_breed: {e}")
        return {"error": str(e)}

# ----------------------------
# Flask API
# ----------------------------
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    f = request.files["file"]
    if f.filename == '':
        return jsonify({"error": "No file selected"}), 400

    filepath = os.path.join("uploads", f.filename)
    os.makedirs("uploads", exist_ok=True)
    f.save(filepath)

    try:
        result = predict_breed(filepath)
        # Clean up uploaded file
        if os.path.exists(filepath):
            os.remove(filepath)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

# For Dog adaptation

In [ ]:
import os
import numpy as np
import cv2
from flask import Flask, request, jsonify
from PIL import Image
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input  # Add this import
import tensorflow as tf

print(tf.__version__)

# ----------------------------
# [INFO] Load models
# ----------------------------
print("[INFO] Loading YOLO model...")
yolo = YOLO("yolov8n.pt")

print("[INFO] Loading custom breed model...")
classifier = load_model("stanford_dogs_model.keras")

with open("standfordDogClasses.txt") as f:
    BREED_CLASSES = [line.strip() for line in f]

# ----------------------------
# Helper function to generate dog links
# ----------------------------
def get_dog_links(breed_name):
    """Generate links for buying/adopting the predicted dog breed"""
    # Clean breed name for URL (replace underscores with spaces, handle special characters)
    clean_breed = breed_name.replace("_", " ").replace("-", " ").title()
    url_breed = breed_name.replace("_", "%20").replace(" ", "%20")

    links = {
        "adoption": {
            "petfinder": f"https://www.petfinder.com/dog-breeds/{breed_name.lower().replace('_', '-')}/",
            https://www.petfinder.com/search/dogs-for-adoption/?breed[]=Golden%20Retriever
            "adopt_a_pet": f"https://www.adoptapet.com/dog-adoption/breed/{url_breed}",
            "rescue_groups": f"https://www.akc.org/dog-breeds/{breed_name.lower().replace('_', '-')}/rescue-groups/"
        },
        "purchase": {
            "akc_marketplace": f"https://marketplace.akc.org/puppies/browse?breed={url_breed}",
            "puppyfind": f"https://www.puppyfind.com/breed/{breed_name.lower().replace('_', '')}",
            "nextdaypets": f"https://www.nextdaypets.com/{breed_name.lower().replace('_', '')}-puppies-for-sale"
        },
        "breed_info": {
            "akc_info": f"https://www.akc.org/dog-breeds/{breed_name.lower().replace('_', '-')}/",
            "dogtime": f"https://dogtime.com/dog-breeds/{breed_name.lower().replace('_', '-')}",
            "wikipedia": f"https://en.wikipedia.org/wiki/{clean_breed}"
        }
    }

    return links

# ----------------------------
# Helper function
# ----------------------------
def predict_breed(img_path):
    try:
        # Load image as PIL Image first (like in working code)
        img_pil = Image.open(img_path).convert("RGB")
        img_array = np.array(img_pil)

        # Detect dog with YOLO
        results = yolo(img_array)

        # Check if any dogs were detected
        if results[0].boxes is None:
            return {"error": "No dogs detected"}

        boxes = results[0].boxes.xyxy.cpu().numpy()
        labels = results[0].boxes.cls.cpu().numpy()

        # Find the first dog (COCO class 16)
        dog_box = None
        for i, box in enumerate(boxes):
            if int(labels[i]) == 16:  # COCO class 16 = dog
                dog_box = box
                break

        if dog_box is None:
            return {"error": "No dog detected"}

        # Crop the dog region
        x1, y1, x2, y2 = map(int, dog_box)

        # Load image with OpenCV for cropping (like in working code)
        img_cv = cv2.imread(img_path)
        h, w, _ = img_cv.shape

        # Ensure coordinates are within bounds
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)

        if x2 <= x1 or y2 <= y1:
            return {"error": "Invalid crop dimensions"}

        dog_crop = img_cv[y1:y2, x1:x2]

        # Save cropped image temporarily
        crop_path = "temp_crop.jpg"
        cv2.imwrite(crop_path, dog_crop)

        # Load and preprocess exactly like in working code
        img = image.load_img(crop_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)  # Use ResNet50 preprocessing instead of /255.0

        # Predict
        preds = classifier.predict(x)
        print("Check prediction---------->", preds)

        # Get top 3 predictions like in working code
        probs = preds[0]
        top_indices = probs.argsort()[-3:][::-1]  # Top 3 predictions

        top_predictions = []
        for idx in top_indices:
            breed_name = BREED_CLASSES[idx]
            confidence = float(probs[idx])
            top_predictions.append({
                "breed": breed_name,
                "confidence": confidence
            })

        print("Check top predictions---------->", top_predictions)

        # Clean up temporary file
        if os.path.exists(crop_path):
            os.remove(crop_path)

        # Get links for the predicted breed
        breed_links = get_dog_links(top_predictions[0]["breed"])

        return {
            "breed": top_predictions[0]["breed"],
            "confidence": top_predictions[0]["confidence"],
            "links": breed_links
        }

    except Exception as e:
        print(f"Error in predict_breed: {e}")
        return {"error": str(e)}

# ----------------------------
# Flask API
# ----------------------------
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    f = request.files["file"]
    if f.filename == '':
        return jsonify({"error": "No file selected"}), 400

    filepath = os.path.join("uploads", f.filename)
    os.makedirs("uploads", exist_ok=True)
    f.save(filepath)

    try:
        result = predict_breed(filepath)
        # Clean up uploaded file
        if os.path.exists(filepath):
            os.remove(filepath)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/breed-links/<breed_name>", methods=["GET"])
def get_breed_links(breed_name):
    """Get adoption/purchase links for a specific breed"""
    try:
        links = get_dog_links(breed_name)
        return jsonify({
            "breed": breed_name,
            "links": links
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/redirect/<link_type>/<breed_name>", methods=["GET"])
def redirect_to_site(link_type, breed_name):
    """Direct redirect to specific adoption/purchase sites"""
    from flask import redirect

    try:
        links = get_dog_links(breed_name)

        # Define redirect mappings
        redirect_map = {
            "petfinder": links["adoption"]["petfinder"],
            "adoptapet": links["adoption"]["adopt_a_pet"],
            "akc-marketplace": links["purchase"]["akc_marketplace"],
            "puppyfind": links["purchase"]["puppyfind"],
            "breed-info": links["breed_info"]["akc_info"]
        }

        if link_type in redirect_map:
            return redirect(redirect_map[link_type])
        else:
            return jsonify({"error": "Invalid link type"}), 400

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)